In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.metrics import brier_score_loss
from scipy.special import logit, expit
from tqdm import tqdm

In [24]:
df = pd.read_csv('hw4_long.csv', parse_dates = ['Date'])
df['GameID'] = df.index

In [53]:
df1517 = df[df.Y.between(15,17)].groupby(['Y','Team'], as_index = False)['GD'].mean()
df1416 = df[df.Y.between(14,16)].groupby(['Y','Team'], as_index = False)['GD'].mean()
df1416.rename({'GD':'GD_Prev'}, inplace = True, axis = 1)
df1416['Y'] = df1416['Y']+1

In [58]:
dftest = df1517.merge(df1416, on = ['Team','Y'], how = 'left')
dftest.dropna(axis = 0, inplace = True)

In [65]:
# 1a
ols_model = smf.ols('GD ~ GD_Prev', dftest).fit()
ols_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     GD   R-squared:                       0.636
Model:                            OLS   Adj. R-squared:                  0.635
Method:                 Least Squares   F-statistic:                     437.5
Date:                Sat, 06 Mar 2021   Prob (F-statistic):           7.70e-57
Time:                        16:12:04   Log-Likelihood:                -155.71
No. Observations:                 252   AIC:                             315.4
Df Residuals:                     250   BIC:                             322.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0337      0.029     -1.162      0.246      -0.091       0.023
GD_Prev        0.8767      0.042     20.915      0.000       0.794       0.959
==============================================================================
Omnibus:                       16.916   Durbin-Watson:                   2.229
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               23.025
Skew:                           0.477   Prob(JB):                     1.00e-05
Kurtosis:                       4.132   Cond. No.                         1.53
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [70]:
# 1b
df1 = pd.read_csv('hw4.csv', parse_dates = ['Date'])
df1 = df1[df1.Y.between(14,17)]

In [71]:
ols_model1 = smf.ols('GD_Home ~ I(logit(pH) - logit(pA)) - 1', df1).fit()
ols_model1.summary()
# Verifying 0.4897 coefficient

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                GD_Home   R-squared (uncentered):                   0.281
Model:                            OLS   Adj. R-squared (uncentered):              0.281
Method:                 Least Squares   F-statistic:                              2852.
Date:                Mon, 08 Mar 2021   Prob (F-statistic):                        0.00
Time:                        14:58:13   Log-Likelihood:                         -13730.
No. Observations:                7304   AIC:                                  2.746e+04
Df Residuals:                    7303   BIC:                                  2.747e+04
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
I(logit(pH) - logit(pA))     0.4897      0.009     53.405      0.000       0.472       0.508
==============================================================================
Omnibus:                       41.901   Durbin-Watson:                   2.037
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               60.249
Skew:                           0.027   Prob(JB):                     8.26e-14
Kurtosis:                       3.442   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [73]:
# 1bi) Converting SD to GD
ols_model2 = smf.ols('GD_Home ~ I(SD_Home - SD_Away) - 1', df1).fit()
ols_model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                GD_Home   R-squared (uncentered):                   0.134
Model:                            OLS   Adj. R-squared (uncentered):              0.134
Method:                 Least Squares   F-statistic:                              1133.
Date:                Mon, 08 Mar 2021   Prob (F-statistic):                   5.09e-231
Time:                        15:01:11   Log-Likelihood:                         -14407.
No. Observations:                7304   AIC:                                  2.882e+04
Df Residuals:                    7303   BIC:                                  2.882e+04
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
I(SD_Home - SD_Away)     0.0410      0.001     33.659      0.000       0.039       0.043
==============================================================================
Omnibus:                       36.348   Durbin-Watson:                   2.015
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               50.932
Skew:                           0.022   Prob(JB):                     8.71e-12
Kurtosis:                       3.407   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [74]:
# 1bii) Converting xGD to GD
ols_model3 = smf.ols('GD_Home ~ I(xGD_Home - xGD_Away) - 1', df1).fit()
ols_model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                GD_Home   R-squared (uncentered):                   0.492
Model:                            OLS   Adj. R-squared (uncentered):              0.492
Method:                 Least Squares   F-statistic:                              7081.
Date:                Mon, 08 Mar 2021   Prob (F-statistic):                        0.00
Time:                        15:03:16   Log-Likelihood:                         -12459.
No. Observations:                7304   AIC:                                  2.492e+04
Df Residuals:                    7303   BIC:                                  2.493e+04
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
I(xGD_Home - xGD_Away)     0.4969      0.006     84.147      0.000       0.485       0.508
==============================================================================
Omnibus:                       31.357   Durbin-Watson:                   2.022
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               42.960
Skew:                          -0.010   Prob(JB):                     4.69e-10
Kurtosis:                       3.375   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""